### Legal Text Summarization

#### Import libraries/packages

In [223]:
import os
import shutil
import textract
import numpy as np

from nltk.tokenize import sent_tokenize, word_tokenize
from nltk import corpus, pos_tag 
import gensim 
import enchant
import re
import math
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

import logging
import pprint

from gensim.models.doc2vec import Doc2Vec, TaggedDocument

#### Configure pretty print

In [2]:
pp = pprint.PrettyPrinter( indent = 4, width = 150 )

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

logger = logging.getLogger()
logger.setLevel(logging.CRITICAL)

#### Get the english dictionary words

In [3]:
dictionary = enchant.Dict( "en_US" )

#### Define all required methods

In [291]:
def get_files_in_path( path ) :
    
    for root, dirs, files in os.walk( path ) :
        
        for filename in files :
            
            yield root + "/" + filename
            
            
def load_text_from_file( filename ) :
    
    contents = textract.process( filename, encoding='ascii' )
    contents = re.sub( '\\n',' ', str( contents ) )
    
    return contents


def delete_dir_contents( path ) :
    
    shutil.rmtree( path )
    
    os.makedirs( path )
    
    
def number_of_files_in_path( path ) :
    
    return len( next( os.walk( path ) )[ 2 ] )


def get_file_extension( filename ) :
    
    return os.path.splitext( filename )[ 1 ]


def get_file_basename( filename ) :
    
    return os.path.basename( filename )


def is_supported_file_type( filename ) :
    
    # A list of all supported extensions
    valid_extensions = [ ".doc", ".docx", ".pdf", ".rtf" ]
    
    return valid_extensions.count( get_file_extension( filename ) ) == 1


def generate_next_filename( path, extension ) :
    
    return path + "/" + str( number_of_files_in_path( path ) + 1 ) + extension


def write_data_to_file( filename, data ) :
    
    # Open the file for writing
    file = open( filename, 'w' )
    
    # Write the data to the file
    file.write( data )
    
    # Close the file
    file.close()
    
    
def generate_raw_files( source_dir, target_dir ) :
    
    # Clear all existing raw cases
    delete_dir_contents( target_dir + "/cases" )

    # Clear all existing raw summaries
    delete_dir_contents( target_dir + "/summaries" )
    
    for filename in get_files_in_path( source_dir ) :
    
        # Get the file extension
        extension = get_file_extension( filename )

        if is_supported_file_type( filename ) :

            # Load the file contents
            contents = load_text_from_file( filename )

            # Generate a new filename based on a numerical sequence
            new_filename = generate_next_filename( target_dir + "/" + ( "summaries" if "1.0" in filename else "cases" ), ".txt" )

            # Write the new filename
            write_data_to_file( new_filename, contents )
    
    
def tokenize_file_contents( filename ) :
    
    # Load the stop words
    stop_words = corpus.stopwords.words('english')
    
    # Load the file contents
    contents = load_text_from_file( filename )
    
    # Define the filename tokens
    content_tokens = []
    
    # Tokenize the sentences from the file contents
    sentence_tokens = sent_tokenize( contents )
    
    for sentence in sentence_tokens :
        
        # Tokenize the words from the sentence
        word_tokens = word_tokenize( sentence )
        
        # Remove the punctuation and convert the words to lowercase
        words = [ word.lower() for word in word_tokens if word.isalpha() and dictionary.check( word ) ]
        
        # Remove the stop words
        words = [ word for word in words if not word in stop_words ]
        
        # Append the sentence and tokens to the list
        content_tokens.append( { 'sentence': sentence, 'words': words } )
        
    return content_tokens


def load_files_in_path( path ) :
    
    # Create a list of tokens
    contents = []
    
    for filename in get_files_in_path( path ) :
        
        # Append a dictionary to the contents
        contents.append( load_text_from_file( filename ) )
        
    return contents



def tokenize_files_in_path( path ) :
    
    # Create a list of tokens
    tokens = []
    
    for filename in get_files_in_path( path ) :
        
        # Append a dictionary to the tokens
        tokens.append( { "filename": get_file_basename( filename ), "tokens": tokenize_file_contents( filename ) } )
        
    return tokens


def create_word2vec_model( entries ) :
    
    # Create documents list
    documents = []
    
    for entry in entries :
        
        # Create a file token
        file_tokens = []
        
        for sentence in entry[ "tokens" ] :
            
            # Add the word tokens in the document
            file_tokens =  file_tokens + sentence[ "words" ]
            
        # Add the file tokens to the document
        documents.append( file_tokens )
    
    model = gensim.models.Word2Vec ( documents, size = 50, window = 5, min_count = 1, workers = 100 )
    
    model.train( documents, total_examples = len( documents ), epochs = 10 )
    
    return model


def get_synonyms( model, word, min_similarity = 0.85 ) :
    
    # Define the similar words
    synonyms = []
    
    # Get related word2vec words
    suggested_words = model.wv.most_similar( positive = word, topn = 5 )
    
    # Limit words to a given similarity
    for word in suggested_words :
        
        if word[ 1 ] > min_similarity :
        
            # Add word to 
            synonyms.append( word[ 0 ] )
            
    return synonyms


def get_tfidf( file_entries ) :
    
    corpus = []
    
    for file_index, file_entry in enumerate( file_entries ) :
        
        document = []
        
        for token_index, token in enumerate( file_entry[ 'tokens' ] ) :
            
            document.append( token[ 'sentence' ] )
            
        corpus.append( " ".join( document ) )
    
    tf = TfidfVectorizer( smooth_idf = False, sublinear_tf = False, norm = None, analyzer = 'word' )
    
    return dict( zip( tfidf[ 1 ].get_feature_names(), tfidf[ 0 ].idf_ ) )


def generate_score( weight, sentence_position, total_sentences, sentence_length, sentence_length_mean, sentence_length_std ) :
    
    position_factor = math.sin( math.radians( ( float( sentence_position ) / total_sentences ) ) )
    
    length_factor = sentence_length_mean / np.square( sentence_length  - ( sentence_length_std * 2 ) )

    return position_factor * length_factor * weight


def score_tokenized_sentences( file_entries, word2vec_model ):
    
    tfidf = get_tfidf( file_entries )
    
    # Iterate through the files
    for file_index, file_entry in enumerate( file_entries ) :
        
        #if file_index < 10 :

            # Define the cleaned sentences
            cleaned_sentences = []
            sentences_lengths = []

            # Iterate through the tokens
            for token_index, token in enumerate( file_entry['tokens' ] ):

                # Add the words to the cleaned sentences
                cleaned_sentences.append( " ".join( str( word ) for word in token['words'] ) )
                sentences_lengths.append( len( token[ 'words' ] ) )

            for token_index, cleaned_sentence in enumerate( cleaned_sentences ) :
                
                # Words
                words = file_entries[ file_index ][ 'tokens' ][ token_index ][ 'words' ]

                # Define the sentence score
                sentence_score = 0

                # Iterate through the bag of words
                for word_index, word in enumerate( words ) :

                    if tfidf.has_key( word ) :
                        
                        # Increment the sentence score by the score
                        sentence_score = sentence_score + tfidf[ word ]

                # Get the sentence tags
                pos_tags = pos_tag( file_entries[ file_index ][ 'tokens' ][ token_index ][ 'words' ] )

                # Define the tags
                tags = []

                for tag in enumerate( pos_tags ) :

                    tags.append( tag[ 1 ][ 1 ] )

                weight = 0 if len( words ) < 1 else float( sentence_score ) / len( words )
                total_sentences = len( cleaned_sentences )
                sentence_position = token_index + 1
                sentence_length = len( file_entries[ file_index ][ 'tokens' ][ token_index ][ 'words' ] )
                sentence_length_mean = np.mean( sentences_lengths )
                sentence_length_std = np.std( sentences_lengths )

                # Create the store dictionary
                score = { 
                    'weight': weight, 
                    'total_sentences': total_sentences, 
                    'sentence_position': sentence_position, 
                    'tense': 'past' if 'VBD' in tags else 'present' if 'VBG' in tags else 'uncategorized',
                    'sentence': file_entries[ file_index ][ 'tokens' ][ token_index ][ 'sentence' ],
                    'rating': generate_score( weight, sentence_position, total_sentences, sentence_length, sentence_length_mean, sentence_length_std ),
                    'sentence_length': sentence_length
                }

                file_entries[ file_index ][ "tokens" ][ token_index ][ "score" ] = score
                

    return file_entries


def get_tokens_minimum_ratings( scored_tokens, number_of_sentences = 5 ) :
    
    token_ratings = []
    
    for entry_index, scored_token in enumerate( scored_tokens ) :
        
        ratings = []
        
        for sentence_index, sentence in enumerate( scored_token[ "tokens" ] ) :
            
            if sentence.has_key( "score" ) :
            
                ratings.append( sentence[ "score" ][ "rating" ] )
                
            else :
                
                ratings.append( 0.0 )
            
        total_ratings = len( ratings )
        
        index = ( number_of_sentences if number_of_sentences < total_ratings else total_ratings ) - 1
        
        token_ratings.append( sorted( ratings, reverse = True )[ index ] )
        
    return token_ratings


def create_summaries( entries, number_of_sentences ) :
    
    summaries = []
    
    # Declare the output directory
    #output_dir = "output"
    
    # Clear all existing outputs
    #delete_dir_contents( output_dir )
    
    # Get minimum ratings
    token_minimum_ratings = get_tokens_minimum_ratings( entries, number_of_sentences )
    
    # Iterate through the tokens
    for entry_index, entry in enumerate( entries ) :
        
        #if entry_index < 10 :

            # Declare the summary sentences list
            #sentences = [ entry[ "tokens" ][ 0 ][ "score" ][ "sentence" ] ] if len( entry[ "tokens" ] ) > 0 else [] 
            sentences = []

            # Iterate through the tokens
            for token_index, token in enumerate( entry[ "tokens" ] ) :

                # Only use sentences that score above the minimum token
                
                if token.has_key( "score" ) :
                    
                    if token[ "score" ][ "rating" ] >= token_minimum_ratings[ entry_index ] :

                        #Add the sentence to the sentences
                        sentences.append( token[ "score" ][ "sentence"] )

            # Generate a new filename based on a numerical sequence
            #new_filename = generate_next_filename( output_dir, ".txt" )

            # Write the new filename
            # write_data_to_file( new_filename, " ".join( sentences ) )

            summaries.append( " ".join( sentences ) )
            
    return summaries

#### Create the raw files

In [5]:
# Generate the raw files
generate_raw_files( "./source", "./target" )

#### Generate tokens

In [155]:
# Cases tokens
tokenized_cases = tokenize_files_in_path( "./target/cases" )

In [156]:
# Summaries tokens
tokenized_summaries = tokenize_files_in_path( "./target/summaries" )

#### Create the word2vec models

In [10]:
# Generate cases word2vec model
cases_word2vec_model = create_word2vec_model( tokenized_cases )

In [11]:
# Generate summaries word2vec model
summaries_word2vec_model = create_word2vec_model( tokenized_summaries )

In [12]:
# Generate general (cases + summaries ) word2vec model
general_word2vec_model = create_word2vec_model( tokenized_cases + tokenized_summaries )

#### Test word2vec models (check similarities)

In [14]:
# Get synonyms based on the cases model
print get_synonyms( cases_word2vec_model, "murder", 0.8 )

['manslaughter', 'convicted', 'provocation', 'rape', 'kidnapping']


In [15]:
# Get synonyms based on the summaries model
print get_synonyms( summaries_word2vec_model, "murder", 0.8 )

['violence', 'robbery', 'count', 'kidnapping', 'intent']


In [16]:
# Get synonyms based on the general model
print get_synonyms( general_word2vec_model, "murder", 0.8 )

['kidnapping', 'manslaughter']


#### Score tokens

In [212]:
# Score the cases
scored_case_tokens = score_tokenized_sentences( tokenized_cases, general_word2vec_model )

In [213]:
# Score the summaries
scored_summary_tokens = score_tokenized_sentences( tokenized_summaries, general_word2vec_model )

#### Created the TF-IDF

In [261]:
case_summaries = create_summaries( scored_case_tokens, number_of_sentences = 15 )

In [271]:
summary_summaries = create_summaries( scored_summary_tokens, number_of_sentences = 15 )

In [272]:
case_summaries[ 0 : 2 ]

["The  defendant then suspected the plaintiff and the two key holders  in  the  loss  of  the said money. He stated that he was  involved  in  the  termination  exercise since he was the Chairman Staff Commendation and Disciplinary Committee. The position of the law in  regard  to  termination  of  any  employee  from employment with or without notice is now beyond any doubt. But this was a matter  which  was  subject  of  a disciplinary proceedings where it was established  that  the  plaintiff  was not connected with the loss of the said dollars. The plaintiff has emphatic that after handing over the said dollars  to  the two key holders, he had nothing to do with its  loss. It was done without  considering  the number of good years the plaintiff  had  put  in  the  defendant's  service. The plaintiff was  one  of  the suspects because he was the one who had brought the money  which  eventually got stolen. It  was connected to loss  of  public  money:   see  Abraham  Waligo   Vs   At

In [273]:
summary_summaries[ 0 : 2 ]

['THE REPUBLIC OF UGANDA IN THE HIGH COURT OF UGANDA AT KAMPALA CIVIL SUIT No. 669 OF 2001 FACTS  The brief facts giving rise to the cause of action are that the plaintiff was appointed in the employment of the defendant on the 15th July 1979. He served in various departments of the bank until 21st September 1999 when at the instance of the bank he was arrested and detained at Kawempe police station. He was arrested because of the loss of some US $200.000 which had been collected form Kasese branch and handed to two officers of the bank. He was released on bond on 27th September 1999 but kept on reporting at a police post at the defendants head office premises in Kampala. He appeared before the Staff Commendation and Disciplinary Committee between January and March 2000 on allegation of the loss of the said money. The committee exonerated him but was however terminated on the 12th June 2000. The other two suspects were however dismissed from service. The plaintiff appealed to the defen

### Evaluate summaries

In [294]:
summaries = load_files_in_path( './target/summaries' )

In [275]:
model = gensim.models.Doc2Vec(vector_size=250, window=2, min_count=5, workers=11,alpha=0.025, min_alpha=0.025)

In [276]:
tagged_data = [TaggedDocument(words=word_tokenize(_d.lower()), tags=[str(i)]) for i, _d in enumerate(case_summaries[0:250])]

In [277]:
max_epochs = 100
vec_size = 20
alpha = 0.025

model = Doc2Vec(size=vec_size,
                alpha=alpha, 
                min_alpha=0.00025,
                min_count=1,
                dm =1)
  
model.build_vocab(tagged_data)

for epoch in range(max_epochs):
    model.train(tagged_data,
                total_examples=model.corpus_count,
                epochs=model.epochs)
    # decrease the learning rate
    model.alpha -= 0.0002
    # fix the learning rate, no decay
    model.min_alpha = model.alpha

model.save("models/cases.model")
print("Model Saved")

Model Saved


In [303]:
test_data = word_tokenize(summaries[ 3 ].lower())
v1 = model.infer_vector(test_data)

In [304]:
model.docvecs.most_similar([v1])

[('114', 0.7531960606575012),
 ('24', 0.6740461587905884),
 ('193', 0.6674911975860596),
 ('29', 0.664217472076416),
 ('66', 0.6601599454879761),
 ('95', 0.6589336395263672),
 ('227', 0.6533979177474976),
 ('237', 0.6516009569168091),
 ('245', 0.6460717916488647),
 ('201', 0.6456685066223145)]